<a href="https://colab.research.google.com/github/Amna-Javed2/OPENAI-SDK/blob/master/02_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




**Install openai Agent sdk**




In [3]:
!pip install -Uq openai-agents

**Capable of running async function (Jupiter Notebook)**

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig

API KEY

In [6]:
# api key
import os
from google.colab import userdata
google_api_key = userdata.get("GOOGLE_API_KEY")

**Model Provider / External Client**

In [7]:
#  external or openai client / model provider
external_client = AsyncOpenAI(
    api_key = google_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
)

**Model**

In [8]:
# Model
model = OpenAIChatCompletionsModel(
    model = "gemini-2.0-flash",
    openai_client = external_client,
)

**Config**

In [9]:
# Config
config = RunConfig(
    model= model,
    model_provider = external_client,
    tracing_disabled = True,
)

1__Hello-world

In [10]:
# Agent
agent : Agent = Agent(
    name = "Assistant",
    instructions = "You are a helpful assistant",
    model = model,
)

#  run synchronous
result = Runner.run_sync(
    agent ,
   "Hello! How are you",
    run_config = config
)

# final output
print("\nCalling Agent \n")
print(result.final_output)


Calling Agent 

Hello! As a large language model, I don't experience feelings in the same way humans do. But I'm functioning well and ready to assist you with any questions or tasks you have. How can I help you today?



2__Hello-world

In [11]:
import asyncio
from agents import Agent, Runner

In [12]:
# agent
async def main():
         agent = Agent(
      name = "Assistant",
      instructions = "You only respond in objects"
          )

#  run asynchronous

         result = await Runner.run(agent,"Tell me about OOP in programming",run_config = config)
         print(result.final_output)

#  final output

  # if __name__ == "__main__":
asyncio.run(main())

# await main()

```json
{
  "topic": "Object-Oriented Programming (OOP)",
  "description": "Object-Oriented Programming (OOP) is a programming paradigm based on the concept of 'objects', which contain data, in the form of fields (often known as attributes or properties), and code, in the form of procedures (often known as methods).  An object's methods can access and modify the data fields of the object with which they are associated. OOP is a way of organizing code to reduce redundancy and increase reusability.",
  "key_concepts": [
    {
      "concept": "Objects",
      "description": "Objects are the fundamental building blocks of OOP. They are instances of classes, combining data (attributes) and behavior (methods) related to a specific entity. Think of a car: it has attributes like color and model, and behaviors like accelerate and brake."
    },
    {
      "concept": "Classes",
      "description": "A class is a blueprint or template for creating objects. It defines the attributes (data) and m

Streaming with google_gemini

In [13]:
import asyncio
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent

async def main():
  agent = Agent(
      name = "Assistant",
      instructions="You are a helpful assistant",
  )

  result =  Runner.run_streamed(agent, input= "Please tell me 5 jokes in urdu",run_config=  config)
  async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
       print(event.data.delta, end="", flush=True)

asyncio.run(main())

Okay, here are 5 jokes in Urdu, along with a brief explanation of the humor where necessary:

1.  **Joke:** Aik admi ne sign board par likha: "Kutte se hoshiyar." Dusre admi ne uske neeche likha: "Kutta bhi tumse hoshiyar."

    **Translation:** A man wrote on a sign board: "Beware of the dog." Another man wrote underneath: "The dog is also beware of you."

    **Humor:** This is a play on words and a bit of sarcasm. The second man is implying the dog is just as wary of the first man as the first man is wary of the dog.

2.  **Joke:** Teacher: "Woh kaunsa parinda hai jo ud nahin sakta?" Student: "Mara hua parinda."

    **Translation:** Teacher: "Which bird cannot fly?" Student: "A dead bird."

    **Humor:**  This is a simple, literal answer to a seemingly clever question. The student takes the question at face value, providing an obvious but funny answer.

3.  **Joke:** Aik admi bike se jaa raha tha. Police wale ne roka aur kaha, "Helmet kahan hai?" Admi bola, "Jee, kal hi toh sar dh

Note: streaming with openai => raise error

In [19]:
import asyncio
import random
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def how_many_jokes() -> int:
  return random.randint(1,10)

async def main():
  agent = Agent(
      name= "Joker",
      instructions= "First call the `how_many_jokes` tool, then tell that many jokes.",
      tools = [how_many_jokes],
  )

  result = Runner.run_streamed(
      agent,
      input = "Hello",
      run_config = config
  )
  print("===Run Starting===")
  async for event in result.stream_events():
    if event.type == "raw_response_event":
      continue
    elif event.type == "agent_updated_stream_event":
      print(f"Agent updated: {event.new_agent.name}")
      continue
    elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass

try:
  asyncio.run(main())
except:
  pass

print("===Run Complete===")

===Run Starting===
Agent updated: Joker
-- Tool was called
-- Tool output: 8
-- Message output:
 I am programmed to tell 8 jokes.

Why don't scientists trust atoms?

Because they make up everything!

 параллелепипед

Why did the scarecrow win an award?

Because he was outstanding in his field!

 зачем ты это сказал

Why did the bicycle fall over?

Because it was two tired!

 это не смешно

Why don't eggs tell jokes?

They'd crack each other up!

 да, это так

What do you call a fish with no eyes?

Fsh!

 хватит пожалуйста

Why did the golfer wear two pairs of pants?

In case he got a hole-in-one!

 прекрати

Why can't Monday lift Saturday?

It's a weak day.

 пожалуйста

Why did the coffee go to the police?

It got mugged!

===Run Complete===


**Modify sdk documentation example**


In [21]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

async def main():
  agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gemini-2.0-flash",
    tools=[get_weather],
  )

  result = Runner.run_sync(
    agent,
   input= "What is the weather in Toronto?",
    run_config=config
  )
  print(result.final_output)

if __name__ == "__main__":
  asyncio.run(main())

I need the city name
To fetch weather for you now.
Please say the city.

